# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [13]:
# 1. Download Fashion MNIST Dataset Files
train_image_url = 'https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz?raw=true'
train_label_url = 'https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-labels-idx1-ubyte.gz?raw=true'
test_image_url  = 'https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/t10k-images-idx3-ubyte.gz?raw=true'
test_label_url  = 'https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/t10k-labels-idx1-ubyte.gz?raw=true'

!wget $train_image_url -O train-images-idx3-ubyte.gz
!wget $train_label_url -O train-labels-idx1-ubyte.gz
!wget $test_image_url  -O t10k-images-idx3-ubyte.gz
!wget $test_label_url  -O t10k-labels-idx1-ubyte.gz


--2025-04-20 21:42:15--  https://github.com/zalandoresearch/fashion-mnist/blob/master/data/fashion/train-images-idx3-ubyte.gz?raw=true
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/zalandoresearch/fashion-mnist/raw/refs/heads/master/data/fashion/train-images-idx3-ubyte.gz [following]
--2025-04-20 21:42:16--  https://github.com/zalandoresearch/fashion-mnist/raw/refs/heads/master/data/fashion/train-images-idx3-ubyte.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/refs/heads/master/data/fashion/train-images-idx3-ubyte.gz [following]
--2025-04-20 21:42:16--  https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/refs/heads/master/data/fashion/train-images-idx3-ubyte.gz
Resolving raw.githubusercontent.com 

In [14]:
# 2. Create Custom DataLoader

import gzip
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Reads image file and normalizes pixel values
def load_images(path):
    with gzip.open(path, 'rb') as f:
        f.read(16)
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(-1, 28*28).astype(np.float32) / 255.0

# Reads label file
def load_labels(path):
    with gzip.open(path, 'rb') as f:
        f.read(8)
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

# Custom dataset class
class FashionMNISTCustom(Dataset):
    def __init__(self, image_path, label_path):
        self.images = load_images(image_path)
        self.labels = load_labels(label_path)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.images[idx]), torch.tensor(self.labels[idx]).long()


In [15]:
# 3. Define the Neural Network

import torch.nn as nn
import torch.nn.functional as F

class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [16]:
# 4. Train the Neural Net

# Load training data
train_dataset = FashionMNISTCustom("train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz")
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FashionNet().to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 491.7039
Epoch 2, Loss: 346.7895
Epoch 3, Loss: 311.9561
Epoch 4, Loss: 291.9931
Epoch 5, Loss: 273.5618
Epoch 6, Loss: 257.7265
Epoch 7, Loss: 244.9016
Epoch 8, Loss: 233.6997
Epoch 9, Loss: 224.2854
Epoch 10, Loss: 215.9024


In [17]:
# 5. Save the Weights

# Save trained model weights
torch.save(model.state_dict(), "fashion_model.pth")

# Download the weights file
from google.colab import files
files.download("fashion_model.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# 6. Evaluate Saved Model

# Load test data
test_dataset = FashionMNISTCustom("t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz")
test_loader = DataLoader(test_dataset, batch_size=64)

# Load model and saved weights
model = FashionNet().to(device)
model.load_state_dict(torch.load("fashion_model.pth"))
model.eval()

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 88.19%
